In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-data/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-data/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-data/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-data/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [5]:
# Assuming your CSV file is named 'example.csv' in the extracted folder
df = pd.read_csv('/kaggle/input/llm-data/llm-detect-ai-generated-text/train_essays.csv')

In [6]:
df.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [7]:
len(df)

1820

In [8]:
len(df[df.generated==0])

1375

In [9]:
len(df[df.generated==1])

445

In [10]:
cv = CountVectorizer()

In [11]:
df_train, df_dev = train_test_split(df, test_size = 0.2, random_state = 10)

In [12]:
df_train.head()

,id,prompt_id,text,generated
203,27eaf91f,1,"dear state senator, As you know America is ver...",0
1062,bfe1f62c,1,The presidential Elections.\n\nThe elections a...,0
684,7eb57bec,0,All of these articles share the central idea o...,0
833,9674a0c4,1,The electoral college has many reasons it shou...,0
124,19fb1cce,0,"Every day people wake up, get ready, and go to...",0


In [13]:
from nltk import word_tokenize
# Threshold is set to 5
vocab = {}
vectorizer = CountVectorizer(tokenizer=word_tokenize, min_df=6)

X = vectorizer.fit_transform(df_train['text'])

feature_names = vectorizer.get_feature_names_out()

for feature, count in zip(feature_names, X.sum(axis=0).A1):
    vocab[feature] = count
    
print(vocab)
word2idx = {w: i for i, w in enumerate(vocab)}

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'!': 369, '%': 304, '&': 11, "'": 183, "''": 4240, "'d": 61, "'it": 13, "'ll": 68, "'m": 255, "'pedestrian": 6, "'re": 229, "'s": 2459, "'swing": 9, "'ve": 64, ',': 27083, '.': 28871, '..': 11, '...': 662, '1': 241, '1.': 105, '10': 45, '10.': 22, '100': 18, '11': 26, '11.': 12, '114': 36, '118': 61, '12': 188, '12.': 9, '13': 14, '13.': 18, '14': 30, '14.': 23, '147': 52, '15': 23, '15.': 11, '16': 35, '16.': 9, '1639': 8, '17': 23, '18': 19, '1888': 13, '1888.': 6, '19': 24, '19.': 12, '1960': 105, '1960.': 14, '1968': 54, '1976': 26, '1990': 7, '1990s': 9, '1992': 30, '1995': 20, '1995.': 15, '2': 347, '2.': 80, '20': 44, '20.': 13, '200': 11, '2000': 322, '2000.': 17, '2001': 49, '2005': 45, '2005.': 7, '2006': 16, '2009': 33, '2009.': 21, '2012': 58, '2013': 37, '2014': 8, '21': 22, '21.': 9, '22': 25, '22.': 7, '22euro': 58, '23': 58, '23.': 6, '23rd': 9, '24': 16, '24.': 9, '25': 96, '25.': 13, '266': 7, '266.': 7, '269': 6, '27': 13, '270': 66, '271': 16, '28.': 10, '29': 32, 

In [14]:
def preprocess_data(data_row):
    print(data_row)
    stop_words = set(stopwords.words('english'))
    tokens = [word.lower() for word in nltk.word_tokenize(data_row['text']) if word.isalpha() and word.lower() not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    
    return stemmed_tokens

In [15]:
df.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [16]:
df_train['tokens'] = df.apply(preprocess_data, axis=1)
df_train.head()

id                                                    0059830c
prompt_id                                                    0
text         Cars. Cars have been around since they became ...
generated                                                    0
Name: 0, dtype: object
id                                                    005db917
prompt_id                                                    0
text         Transportation is a large necessity in most co...
generated                                                    0
Name: 1, dtype: object
id                                                    008f63e3
prompt_id                                                    0
text         "America's love affair with it's vehicles seem...
generated                                                    0
Name: 2, dtype: object
id                                                      940276
prompt_id                                                    0
text         How often do you ride in a car? Do y

,id,prompt_id,text,generated,tokens
203,27eaf91f,1,"dear state senator, As you know America is ver...",0,"[dear, state, senat, know, america, differ, co..."
1062,bfe1f62c,1,The presidential Elections.\n\nThe elections a...,0,"[presidenti, elect, elect, brutal, time, candi..."
684,7eb57bec,0,All of these articles share the central idea o...,0,"[articl, share, central, idea, minim, pollut, ..."
833,9674a0c4,1,The electoral college has many reasons it shou...,0,"[elector, colleg, mani, reason, stay, reason, ..."
124,19fb1cce,0,"Every day people wake up, get ready, and go to...",0,"[everi, day, peopl, wake, get, readi, go, work..."


In [17]:
essays = len(df_train['text'])
human_essays = len(df_train[df_train['generated'] == 0])
llm_essays = len(df_train[df_train['generated'] == 1])
p_human = human_essays / essays
p_llm = llm_essays / essays
print(p_human)
print(p_llm)

0.7589285714285714
0.24107142857142858


In [18]:
word_counts_human = np.zeros(len(vocab), dtype=int)
word_counts_llm = np.zeros(len(vocab), dtype=int)
for index, row in df_train.iterrows():
    tokens = row["tokens"]
    if row["generated"] == 0:
        word_counts_human[[word2idx[w] for w in tokens if w in vocab]] += 1
    else:
        word_counts_llm[[word2idx[w] for w in tokens if w in vocab]] += 1

In [19]:
np.set_printoptions(threshold=np.inf)
print(word_counts_human)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 181   0   0   0   0   0   0   0  13   0   0   0  67   0
   0   6   0  18   0  38   0   0   0   0   0   0  82  58  82   0   0   0
   0   0 276   0 111  12  38   0   0   0   0   0  18   0   0  32   0   0
   0   0   0   0   0   0   0   0  10   0  19 122   0   0   0  54  23  11
   0   0   0  68   0   0  43   0   0   7   0   0  12   9  15 459 116  10
   0   0   0   0   0 325   0   0   0 217   0 166  4

In [20]:
print(word_counts_llm)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  15   0   0   0   0   0   0  15   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   4   1   0   0   0
   0   0   1   0   0   2   1   0   0   0   0   0  19   0   0   3   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   2   0   0   0   0   0   0   0   0   0   1   0   6   0   0
   0   0   0   0   0   2   0   0   0   0   0   0   

In [21]:
p_word_human = {}
p_word_llm = {}
def calculate_probabilities(alpha):
    global p_word_human, p_word_llm
    p_word_human = (word_counts_human + alpha) / (human_essays + alpha * len(vocab))
    p_word_llm = (word_counts_llm + alpha) / (llm_essays + alpha * len(vocab))
    return p_word_human, p_word_llm

In [22]:
def predict(essay, alpha):
    p_word_human, p_word_llm = calculate_probabilities(alpha)
    words = essay.lower().split()

    log_p_human = np.log(p_human) + np.sum(np.log(p_word_human[[word2idx[w] for w in words if w in vocab]]))
    log_p_llm = np.log(p_llm) + np.sum(np.log(p_word_llm[[word2idx[w] for w in words if w in vocab]]))

    # Return the predicted class
    if log_p_human > log_p_llm:
        return 0
    else:
        return 1
        

In [23]:
def evaluate(df, alpha):
    y_true = df["generated"]
    y_pred = df["text"].apply(predict, alpha = alpha)
    
    # Calculate accuracy
    acc_score = accuracy_score(y_true, y_pred)
    
    return acc_score
    
# Evaluate the NBC on the development set
alpha_values = [0.1, 0.3, 0.5, 0.8, 1, 1.3, 1.5]
best_accuracy = 0
best_alpha = 0
p_word_human = {}
p_word_llm = {}
for alpha in alpha_values:
    acc_score = evaluate(df_dev, alpha)
    if(acc_score > best_accuracy):
        best_accuracy = acc_score
        best_alpha = alpha
print(f"Best Accuracy_score: {best_accuracy:.2f}")
print(f"Alpha: {best_alpha}")

Best Accuracy_score: 0.90
Alpha: 0.1


In [24]:
# Get top words based on p_word_human and p_word_llm
top_words_human_indices = np.argsort(-p_word_human)[:10]
top_words_llm_indices = np.argsort(-p_word_llm)[:10]

# Retrieve the actual words from vocab
top_words_human = [word for word, index in vocab.items() if index in top_words_human_indices]
top_words_llm = [word for word, index in vocab.items() if index in top_words_llm_indices]

print("Top 10 words that contribute to predicting human generated text: ", top_words_human)
print("Top 10 words that contribute to predicting LLM generated text: ", top_words_llm)


Top 10 words that contribute to predicting human generated text:  ['next', 'voted']
Top 10 words that contribute to predicting LLM generated text:  []


In [25]:
df_testEssays = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [26]:
test_pred = df_testEssays["text"].apply(predict, alpha = best_alpha)

In [27]:
submission_df = pd.DataFrame({'id': df_testEssays['id'], 'generated': test_pred})

In [28]:
submission_df.to_csv('submission.csv', index = False)